In [3]:
import pandas as pd
from googleapiclient.discovery import build
import scrapetube

In [4]:
api_key = 'api_key'
youtube = build('youtube', 'v3', developerKey=api_key)

search_response = youtube.search().list(
    q = '삼프로tv',
    order = 'relevance',
    part = 'snippet',
    maxResults = 10
    ).execute()

channel_id = search_response['items'][0]['snippet']['channelId']

In [5]:
channel_id

'UChlv4GSd7OQl3js-jkLOnFA'

In [18]:
videos = scrapetube.get_channel(channel_id)

video_id = []

for video in videos:
    video_id.append(video['videoId'])

In [19]:
len(video_id)

8402

In [20]:
video_infos = []

for i in range(100):
    start = i * 50 
    end = (i + 1) * 50
    video_request = youtube.videos().list(
        part="snippet",
        id=','.join(video_id[start:end]))

    video_response = video_request.execute()

    for item in video_response['items']:
        title = item['snippet']['title']
        if ('글로벌 이슈체크' in title) or ('글로벌 마켓브리핑' in title) or ('직장인 vlog' in title):
            continue
        video_infos.append([item['snippet']['title'], item['snippet']['publishedAt'], item['id']])
        
df = pd.DataFrame(video_infos, columns=['title', 'date', 'id'])
df.head()

,title,date,id
0,"[오늘아침 라이브] '경기 침체' 반응하는 원자재 시장, 시사점은? _22.07.0...",2022-07-06T21:16:05Z,Ff2H4mSCbVY
1,드라마 '시맨틱에러' 주목받는 이유/2차 전지 어디로 가나?/기초연금 이대로두면 큰...,2022-07-06T14:09:19Z,7NgufNQUjw8
2,[글로벌 개장시황] 2분기 연속 마이너스 성장 예상... 이미 경기 침체? f. 미...,2022-07-06T14:00:00Z,_HiPgNiUjTo
3,"[심층인터뷰] 하반기 2차전지 투자, 가장 매력적인 종목은? f. 하나증권 김현수 ...",2022-07-06T11:15:59Z,S3uRmcC21z4
4,"[ETF마켓워치] 하반기 주식, 정프로 반대로 해라! f. 삼성자산운용 최창규 본부장",2022-07-06T11:15:29Z,3Vh6lN8SNsI


In [21]:
len(df)

4506

In [22]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

In [23]:
df

,title,date,id
0,"[오늘아침 라이브] '경기 침체' 반응하는 원자재 시장, 시사점은? _22.07.0...",2022-07-06 21:16:05+00:00,Ff2H4mSCbVY
1,드라마 '시맨틱에러' 주목받는 이유/2차 전지 어디로 가나?/기초연금 이대로두면 큰...,2022-07-06 14:09:19+00:00,7NgufNQUjw8
2,[글로벌 개장시황] 2분기 연속 마이너스 성장 예상... 이미 경기 침체? f. 미...,2022-07-06 14:00:00+00:00,_HiPgNiUjTo
3,"[심층인터뷰] 하반기 2차전지 투자, 가장 매력적인 종목은? f. 하나증권 김현수 ...",2022-07-06 11:15:59+00:00,S3uRmcC21z4
4,"[ETF마켓워치] 하반기 주식, 정프로 반대로 해라! f. 삼성자산운용 최창규 본부장",2022-07-06 11:15:29+00:00,3Vh6lN8SNsI
...,...,...,...
4501,[주식대학 미리보기] 김준성 연구위원 - 자동차 시장을 보는 새로운 시각과 밸류에이션,2021-01-22 10:05:04+00:00,lXn8sWW-qiA
4502,삼성전자의 인텔 파운드리 이슈.. 확실해지기 전까지 냉정하게 판단하자 f.이베스트투...,2021-01-22 01:29:53+00:00,JLC74bfKT_E
4503,"전기차 산업, 방향성이 아닌 속도의 문제.. 우리가 확인해야 할 포인트는? f.메리...",2021-01-22 01:18:45+00:00,_neFTZYCG2I
4504,바이든 미국 대통령 취임에 시장이 화답한 한 주 f.삼성선물 전승지 책임연구위원,2021-01-22 01:17:22+00:00,SX0eJZA31mg


In [24]:
df = df.set_index('date')

In [25]:
df_21 = df[(pd.DatetimeIndex(df.index).year == 2021) & (pd.DatetimeIndex(df.index).month >= 6)]
df_22 = df[(pd.DatetimeIndex(df.index).year == 2022) & (pd.DatetimeIndex(df.index).month <= 6)]

In [26]:
df_1y = pd.concat([df_22, df_21])
df_1y

,title,id
date,,
2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk
2022-06-30 14:04:45+00:00,[글로벌 개장시황] 유가 하락 전망하는 3가지 이유 f. 미국주식에미치다 장우석 부사장,7SsI_15R4z4
2022-06-30 12:05:00+00:00,[심층인터뷰] 하락장에서도 버티는 제약·바이오의 비밀 f. 현대차증권 엄민용 책임연구원,lOkhqyvmkC4
2022-06-30 12:04:47+00:00,"[증시셔터맨] 상승하는 편의점과 마트, 더 갈 수 있나? f. IBK투자증권 박근형 부장",cOwbFklaWJs
2022-06-30 02:14:32+00:00,[심층인터뷰] 하락장은 이렇게 진행됩니다 f.김한진 이코노미스트,I-TeO8bJKfQ
...,...,...
2021-06-01 01:41:53+00:00,[증시 염탐정] 기관·외국인 매수 출발.. 종가에는 어떨까? f.이베스트투자증권 염...,K16i24lDTGw
2021-06-01 01:26:30+00:00,"[심층 인터뷰] 미국에서의 현지 생산.. 점유율과 실적, 동반 상승한다? f.이베스...",UOeGThyaVNA
2021-06-01 01:11:52+00:00,[제길선생의 미국 돋보기] 테이퍼링 분수령 될 6월.. 시장은 박스권을 유지할까? ...,524lLbP7z1k


In [43]:
# comments = []

# for video_id in df_1y['id']:
#     api_obj = build('youtube', 'v3', developerKey=api_key)
#     response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, maxResults=100).execute()
 
#     while response:
#         for item in response['items']:
#             comment = item['snippet']['topLevelComment']['snippet']
#             comments.append([comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])

# #             if item['snippet']['totalReplyCount'] > 0:
# #                 for reply_item in item['replies']['comments']:
# #                     reply = reply_item['snippet']
# #                     comments.append([reply['textDisplay'], reply['authorDisplayName'], reply['publishedAt'], reply['likeCount']])
 
#         if 'nextPageToken' in response:
#             response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
#         else:
#             break

# df = pd.DataFrame(comments)
# df.to_excel('sampro_06.xlsx', header=['comment', 'author', 'date', 'num_likes'], index=None)

In [61]:
comments_21 = []

for video_id in df_21['id']:
    api_obj = build('youtube', 'v3', developerKey=api_key)
    response = api_obj.commentThreads().list(part='snippet', videoId=video_id, maxResults=100).execute()
 
    while response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments_21.append([video_id, comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])

        if 'nextPageToken' in response:
            response = api_obj.commentThreads().list(part='snippet', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
        else:
            break

df0 = pd.DataFrame(comments_21, columns=['id', 'comment', 'author', 'comment_date', 'num_likes'])
print(len(df0))
df0.head()

178458


,id,comment,author,comment_date,num_likes
0,41txImmP8BM,새해복많이받으시고 성투합시다우리!,조바이든,2022-01-02T22:52:47Z,1
1,41txImmP8BM,삼프로 tv 새해 복많이 받으세요~~<br>BTS와 이찬원 초청하면 좋겠습니다,강수자,2022-01-02T22:16:09Z,0
2,41txImmP8BM,죄송한데 노래 좀 안나왔으면 ~~~,이은정,2022-01-02T15:59:06Z,1
3,41txImmP8BM,세 분 표정이 넘 따뜻해요~^^ 아빠 미소로 따뜻한 위로 되셨을것 같아요♡,지혜로운자,2022-01-02T11:12:59Z,1
4,41txImmP8BM,"예~ㄱ니, 멋지네요!",김형숙,2022-01-02T07:57:12Z,1


In [63]:
df_21.head()

,video_date,title,id
0,2021-12-31 23:00:03+00:00,[고민상담] 애널리스트 목표가 상향시 따라사기 괜찮을까요? (박병창 부장 & 최예근...,41txImmP8BM
1,2021-12-31 00:00:15+00:00,삼프로PD 맘대로 뽑은 2021 삼프로TV 어워드!,q1utzJ_aoBM
2,2021-12-30 23:00:04+00:00,이것이 라이브다! 삼프로PD 픽 2021 라이브 모먼트!,A6XOg5By9Fc
3,2021-12-30 14:13:41+00:00,[글로벌 개장시황] 장엔젤과 미리보는 2022년 1월 전망 f. 미국주식에미치다 장...,xB3rQPP1VgA
4,2021-12-30 14:10:22+00:00,"호피한 맥주맛이군. 형형색색 수제맥주 시장/패하지 않는 투자, 2022년은 이렇게/...",2kZsiasPSn4


In [66]:
sam_21 = pd.merge(df_21, df0, on='id', how='outer')
sam_21.head()

,video_date,title,id,comment,author,comment_date,num_likes
0,2021-12-31 23:00:03+00:00,[고민상담] 애널리스트 목표가 상향시 따라사기 괜찮을까요? (박병창 부장 & 최예근...,41txImmP8BM,새해복많이받으시고 성투합시다우리!,조바이든,2022-01-02T22:52:47Z,1
1,2021-12-31 23:00:03+00:00,[고민상담] 애널리스트 목표가 상향시 따라사기 괜찮을까요? (박병창 부장 & 최예근...,41txImmP8BM,삼프로 tv 새해 복많이 받으세요~~<br>BTS와 이찬원 초청하면 좋겠습니다,강수자,2022-01-02T22:16:09Z,0
2,2021-12-31 23:00:03+00:00,[고민상담] 애널리스트 목표가 상향시 따라사기 괜찮을까요? (박병창 부장 & 최예근...,41txImmP8BM,죄송한데 노래 좀 안나왔으면 ~~~,이은정,2022-01-02T15:59:06Z,1
3,2021-12-31 23:00:03+00:00,[고민상담] 애널리스트 목표가 상향시 따라사기 괜찮을까요? (박병창 부장 & 최예근...,41txImmP8BM,세 분 표정이 넘 따뜻해요~^^ 아빠 미소로 따뜻한 위로 되셨을것 같아요♡,지혜로운자,2022-01-02T11:12:59Z,1
4,2021-12-31 23:00:03+00:00,[고민상담] 애널리스트 목표가 상향시 따라사기 괜찮을까요? (박병창 부장 & 최예근...,41txImmP8BM,"예~ㄱ니, 멋지네요!",김형숙,2022-01-02T07:57:12Z,1


In [67]:
sam_21.to_csv('sampro_21_2.csv', header=['video_date', 'title', 'id', 'comment', 'author', 'coment_date', 'num_likes'], index=None)

In [28]:
comments_22 = []

for video_id in df_22['id']:
    api_obj = build('youtube', 'v3', developerKey=api_key)
    response = api_obj.commentThreads().list(part='snippet', videoId=video_id, maxResults=100).execute()
 
    while response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments_22.append([video_id, video_date, comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])
 
        if 'nextPageToken' in response:
            response = api_obj.commentThreads().list(part='snippet', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
        else:
            break

df1 = pd.DataFrame(comments_22, columns=['id', 'comment', 'author', 'comment_date', 'num_likes'])
print(len(df1))
df1.head()

96006

In [ ]:
# 댓글 시간 + 영상 업로드 시간